In [1]:
#niezbędne biblioteki
import gym
import random
import numpy as np
import tensorflow as tf
from keras.models     import Sequential
from keras.layers     import Dense, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from skimage import color
from collections import deque
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
env = gym.make('Enduro-v0')
env.reset()
#obs=env.reset()
#preprocess_frame(obs)

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [ 24,  26, 167],
        [ 24,  26, 167],
        [ 24,  26, 167]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [ 24,  26, 167],
        [ 24,  26, 167],
        [ 24,  26, 167]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [ 24,  26, 167],
        [ 24,  26, 167],
        [ 24,  26, 167]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]

In [55]:
#do zobaczenia informacji jakie zwraca nam gra
def play_a_random_game_first():
    for step_index in range(1500):
        env.render()
        if step_index==0:
            action=random.randint(1,3)
        else:
            action = agent.model.predict(observation)
        observation, reward, done, info = env.step(action)
        observation = preprocess_frame(observation)
        print("Step {}:".format(step_index))
        print("action: {}".format(action))
        print("reward:{}".format(reward))
        print("done: {}".format(done))
        print("info: {}".format(info))
        if done:
            break
env.reset()
env.close()

In [56]:
play_a_random_game_first()

Step 0:
action: 1
reward:0.0
done: False
info: {'ale.lives': 0}


IndexError: arrays used as indices must be of integer (or boolean) type

In [33]:
env.close()

In [3]:
#funkcja do preprocesingu ramek. Ucinam górną część (niesitotne niebo) i zmieniam na czarno-biały obrazek
def preprocess_frame(frame):
    img = color.rgb2gray(frame)
    img=img[50:155,20:140]
    img=img.reshape(-1, img.shape[0], img.shape[1], 1)
    return img

In [46]:
class DQNAgent:
    def __init__(self, state_size):
        self.state_size = state_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Conv2D(30, (3,3), strides=2, input_shape=(105, 120, 1), activation='elu'))
        model.add(MaxPooling2D())
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(3, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randint(1,3)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])+1  # returns action
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * \
                       np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action-1] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [60]:
if __name__ == "__main__":
    # initialize gym environment and the agent
    env = gym.make('Enduro-v0')
    agent = DQNAgent(env)
    episodes=500
    # Iterate the game
    for e in range(episodes):
        score = 0
        # reset state in the beginning of each game
        state = env.reset()
        state = preprocess_frame(state)
        for time_t in range(500):
            # turn this on if you want to render
            # env.render()
            # Decide action
            action = agent.act(state)
            # Advance the game to the next frame based on the action.
            # Reward is 1 for every frame the pole survived
            next_state, reward, done, _ = env.step(action)
            if action==1:
                reward=reward+1
            if reward==-1:
                reward=-30
            if reward==1:
                reward=10
            score=score+reward
            next_state = preprocess_frame(next_state)
            # Remember the previous state, action, reward, and done
            agent.remember(state, action, reward, next_state, done)
            # make next_state the new current state for the next frame.
            state = next_state
            # done becomes True when the game ends
            # ex) The agent drops the pole
            if done or time_t==499:
                # print the score and break out of the loop
                print("episode: {}/{}, score: {}"
                      .format(e, episodes, score))
                break
        # train the agent with the experience of the episode
        agent.replay(250)

episode: 0/500, score: 1820.0
episode: 1/500, score: 1640.0
episode: 2/500, score: 1590.0
episode: 3/500, score: 1644.0
episode: 4/500, score: 1432.0
episode: 5/500, score: 1710.0
episode: 6/500, score: 1772.0
episode: 7/500, score: 1742.0
episode: 8/500, score: 1826.0
episode: 9/500, score: 1932.0
episode: 10/500, score: 1880.0
episode: 11/500, score: 1924.0
episode: 12/500, score: 1900.0
episode: 13/500, score: 1920.0
episode: 14/500, score: 1870.0
episode: 15/500, score: 1584.0
episode: 16/500, score: 2042.0
episode: 17/500, score: 1484.0
episode: 18/500, score: 2082.0
episode: 19/500, score: 1760.0
episode: 20/500, score: 2084.0
episode: 21/500, score: 1972.0
episode: 22/500, score: 1902.0
episode: 23/500, score: 1896.0
episode: 24/500, score: 1852.0
episode: 25/500, score: 2302.0
episode: 26/500, score: 2132.0
episode: 27/500, score: 2094.0
episode: 28/500, score: 2122.0
episode: 29/500, score: 1976.0
episode: 30/500, score: 2070.0
episode: 31/500, score: 2122.0
episode: 32/500, s

episode: 260/500, score: 3972.0
episode: 261/500, score: 4086.0
episode: 262/500, score: 4056.0
episode: 263/500, score: 3814.0
episode: 264/500, score: 3994.0
episode: 265/500, score: 4022.0
episode: 266/500, score: 4008.0
episode: 267/500, score: 3906.0
episode: 268/500, score: 3868.0
episode: 269/500, score: 4048.0
episode: 270/500, score: 4084.0
episode: 271/500, score: 4158.0
episode: 272/500, score: 3900.0
episode: 273/500, score: 4018.0
episode: 274/500, score: 4100.0
episode: 275/500, score: 4160.0
episode: 276/500, score: 3990.0
episode: 277/500, score: 4116.0
episode: 278/500, score: 3816.0
episode: 279/500, score: 4138.0
episode: 280/500, score: 4118.0
episode: 281/500, score: 4082.0
episode: 282/500, score: 4058.0
episode: 283/500, score: 4198.0
episode: 284/500, score: 4310.0
episode: 285/500, score: 3852.0
episode: 286/500, score: 3972.0
episode: 287/500, score: 4114.0
episode: 288/500, score: 4070.0
episode: 289/500, score: 4128.0
episode: 290/500, score: 4086.0
episode:

Udało się 0
0
1
Udało się 1
2
Udało się 2
3
4
5
6
7
8
9
Udało się 3
10
11
12
13
14
Udało się 4
15
16
17
Udało się 5
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
Udało się 6
44
45
46
47
48
49
50
51
Udało się 7
52
53
Udało się 8
54
55
56
Udało się 9
57
58
59
60
61
Udało się 10
62
Udało się 11
63
64
65
Udało się 12
66
67
68
69
70
71
72
73
74
75
Udało się 13
76
77
78
79
80
81
82
83
84
85
86
87
Udało się 14
88
Udało się 15
89
Udało się 16
90
91
92
93
94
95
96
97
98
99


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
2846/2846 [==============================] - 31s 11ms/step - loss: 0.3505
Epoch 2/10
2846/2846 [==============================] - 24s 9ms/step - loss: 0.0984
Epoch 3/10
2846/2846 [==============================] - 24s 9ms/step - loss: 0.0934
Epoch 4/10
2846/2846 [==============================] - 24s 9ms/step - loss: 0.0884
Epoch 5/10
2846/2846 [==============================] - 24s 9ms/step - loss: 0.0844
Epoch 6/10
2846/2846 [==============================] - 24s 9ms/step - loss: 0.0797
Epoch 7/10
2846/2846 [==============================] - 24s 9ms/step - loss: 0.0753
Epoch 8/10
2846/2846 [==============================] - 25s 9ms/step - loss: 0.0703
Epoch 9/10
2846/2846 [==============================] - 25s 9ms/step - loss: 0.0663
Epoch 10/10
2846/2846 [==============================] - 25s 9ms/step - loss: 0.0619


In [61]:
#Testowanie
scores = []
#choices = []
for each_game in range(5):
    score = 0
    prev_obs = []
    for step_index in range(5000):
        env.render()
        if len(prev_obs)==0:
            action = random.randint(1,3)
        else:
            action = np.argmax(agent.model.predict(prev_obs))+1
        
        #choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = preprocess_frame(new_observation)
        score+=reward
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:', sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))

[11.0, 22.0, 16.0, 22.0, 30.0]
Average Score: 20.2


NameError: name 'choices' is not defined

In [12]:
#Dla porównania losowe ruchy
scores = []
#choices = []
for each_game in range(20):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        #env.render()
        action = random.randrange(0,8)
        
        #choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=1
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:', sum(scores)/len(scores))

[4464, 4425, 4404, 4410, 4443, 4429, 4433, 4406, 4459, 4447, 4414, 4438, 4484, 4421, 4426, 4438, 4447, 4452, 4463, 4453]
Average Score: 4437.8
